In [1]:
import pandas as pd
from math import comb
from ax.service.ax_client import AxClient
import time
import sys

sys.path.append('../../../0_helper_functions')
import sdlnano as sdl
import calculation as calc

In [2]:
# Design space size calculation


def calculate_combinations(num_variables, total_sum, step):
    steps = int(total_sum / step)
    return comb(steps + num_variables - 1, num_variables - 1)

# Define parameters
step = 0.05
total_sum = 1

# Calculate combinations for each group
count_group1 = calculate_combinations(10, total_sum, step)  # 3 Solid lipids + 3 Polymers + 3 Liquid lipids + 1 Drug
count_group2 = calculate_combinations(4, total_sum, step)   # 3 Surfactants + 1 Water

# Calculate total combinations
total_combinations = count_group1 * count_group2


print("organic design space size: ", round(count_group1/(1e6),2), 'Million')
print("aqueous design space size: ", round(count_group2/(1e3),2), 'Thousand')
print("design space size: ", round(total_combinations/(1e9),2), 'Billion')


organic design space size:  10.02 Million
aqueous design space size:  1.77 Thousand
design space size:  17.74 Billion


# Check the iteration number

In [3]:
iteration = sdl.get_iteration_number()
print("This is the iteration: #", iteration)

This is the iteration: # 0


# Check the drug

In [4]:
drug = 'ACE'
print("This is the drug:", drug)

This is the drug: ACE


# Optimizer initialization

In [5]:
optimizer_init_file_name = "optimizer/optimizer_init.json"

In [6]:
ax_client_init = sdl.initialize_ax()
ax_client_init.save_to_json_file(optimizer_init_file_name)

[WARNING 08-02 07:13:51] ax.service.utils.instantiation: Objective thresholds were not set. They will be selected using a heuristic, but should be specified on the objective for best performance.


# Generate recommendations

In [7]:
time_start = time.time()

# generate recomms
ax_client_init = AxClient.load_from_json_file(optimizer_init_file_name)
unlabeled_trial, ax_client_design = sdl.generate_trials(ax_client=ax_client_init, num_of_trials = 16, drug = drug, bopt=0)

time_end = time.time()

print(round((time_end-time_start)/60), 'min')

[INFO 08-02 07:13:52] ax.service.ax_client: Generated new trial 0 with parameters {'Drug': 48, 'SL_1': 60, 'SL_2': 50, 'SL_3': 32, 'LL_1': 96, 'LL_2': 8, 'LL_3': 10, 'P_1': 34, 'P_2': 85, 'P_3': 76, 'S_1': 89, 'S_2': 88, 'S_3': 81, 'Water': 85} using model Sobol.
[INFO 08-02 07:13:52] ax.service.ax_client: Generated new trial 1 with parameters {'Drug': 93, 'SL_1': 38, 'SL_2': 50, 'SL_3': 84, 'LL_1': 50, 'LL_2': 65, 'LL_3': 78, 'P_1': 53, 'P_2': 26, 'P_3': 20, 'S_1': 36, 'S_2': 4, 'S_3': 29, 'Water': 0} using model Sobol.
[INFO 08-02 07:13:52] ax.service.ax_client: Generated new trial 2 with parameters {'Drug': 51, 'SL_1': 77, 'SL_2': 13, 'SL_3': 24, 'LL_1': 20, 'LL_2': 94, 'LL_3': 27, 'P_1': 92, 'P_2': 2, 'P_3': 28, 'S_1': 16, 'S_2': 30, 'S_3': 59, 'Water': 53} using model Sobol.
[INFO 08-02 07:13:52] ax.service.ax_client: Generated new trial 3 with parameters {'Drug': 9, 'SL_1': 19, 'SL_2': 88, 'SL_3': 73, 'LL_1': 73, 'LL_2': 36, 'LL_3': 60, 'P_1': 20, 'P_2': 63, 'P_3': 75, 'S_1': 58,

0 min


In [8]:
# process the trails into ratios

unlabeled_trial_processed = sdl.process_trails(unlabeled_trial)
unlabeled_trial_processed.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
mean,7.500000,0.099312,0.099125,0.101375,0.097812,0.096938,0.100062,0.102375,0.098687,0.104125,0.100188,0.257125,0.242187,0.255500,0.245188
std,4.760952,0.053876,0.060632,0.061614,0.060009,0.054437,0.058229,0.063210,0.057973,0.065030,0.062024,0.140526,0.128013,0.146817,0.150975
min,0.000000,0.011000,0.017000,0.004000,0.000000,0.000000,0.002000,0.003000,0.004000,0.005000,0.011000,0.019000,0.044000,0.031000,0.000000
25%,3.750000,0.055000,0.052750,0.060250,0.054500,0.047000,0.058250,0.051250,0.065000,0.046750,0.058000,0.152000,0.168000,0.150500,0.163000
50%,7.500000,0.109000,0.093000,0.095000,0.088500,0.101000,0.098500,0.109500,0.085500,0.110500,0.095000,0.262000,0.252000,0.246000,0.233500
75%,11.250000,0.141500,0.148750,0.143750,0.141250,0.129750,0.133000,0.152500,0.140750,0.162250,0.140500,0.354000,0.355500,0.374500,0.326750
max,15.000000,0.176000,0.196000,0.203000,0.217000,0.192000,0.220000,0.198000,0.215000,0.206000,0.247000,0.522000,0.485000,0.473000,0.603000


In [9]:
unlabeled_trial
unlabeled_trial.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.000000,16.000000,16.000000,16.00000,16.000000,16.000000,16.000000,16.000000,16.00000,16.000000,16.000000,16.000000,16.000000
mean,7.500000,50.625000,50.125000,50.187500,49.812500,50.06250,50.562500,50.125000,50.000000,50.000000,49.93750,50.125000,49.875000,50.062500,50.062500
std,4.760952,30.001944,30.001944,30.316593,30.000486,30.12965,30.427989,30.532769,29.926577,30.216993,29.75112,30.046353,29.814706,29.713002,30.049889
min,0.000000,4.000000,5.000000,2.000000,0.000000,0.00000,1.000000,1.000000,2.000000,2.000000,5.00000,5.000000,4.000000,4.000000,0.000000
25%,3.750000,26.750000,26.500000,23.750000,27.750000,27.50000,27.750000,25.750000,26.000000,25.000000,26.00000,26.000000,27.500000,26.500000,29.500000
50%,7.500000,49.500000,48.500000,50.000000,49.000000,51.50000,50.500000,50.000000,50.000000,50.500000,48.50000,52.500000,50.500000,50.500000,48.500000
75%,11.250000,74.500000,75.500000,75.500000,74.750000,74.50000,73.000000,74.250000,72.500000,73.750000,75.25000,72.000000,72.000000,73.500000,74.500000
max,15.000000,100.000000,98.000000,98.000000,95.000000,96.00000,99.000000,100.000000,96.000000,100.000000,97.00000,99.000000,95.000000,97.000000,97.000000


In [10]:
# Convert ratios to volumes

transfer = calc.converter(unlabeled_trial_processed)
transfer.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Solvent
count,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.00000,16.000000,16.000000,16.000000,16.00000,16.000000,16.000000,16.00000,16.000000
mean,7.500000,34.759375,34.693750,35.481250,34.234375,33.928125,35.021875,35.83125,34.540625,36.443750,35.065625,257.12500,242.187500,255.500000,245.18750,0.000000
std,4.760952,18.856619,21.221231,21.564801,21.003196,19.052978,20.380088,22.12335,20.290632,22.760542,21.708360,140.52562,128.012613,146.816893,150.97515,0.338132
min,0.000000,3.850000,5.950000,1.400000,0.000000,0.000000,0.700000,1.05000,1.400000,1.750000,3.850000,19.00000,44.000000,31.000000,0.00000,-0.700000
25%,3.750000,19.250000,18.462500,21.087500,19.075000,16.450000,20.387500,17.93750,22.750000,16.362500,20.300000,152.00000,168.000000,150.500000,163.00000,-0.087500
50%,7.500000,38.150000,32.550000,33.250000,30.975000,35.350000,34.475000,38.32500,29.925000,38.675000,33.250000,262.00000,252.000000,246.000000,233.50000,0.000000
75%,11.250000,49.525000,52.062500,50.312500,49.437500,45.412500,46.550000,53.37500,49.262500,56.787500,49.175000,354.00000,355.500000,374.500000,326.75000,0.087500
max,15.000000,61.600000,68.600000,71.050000,75.950000,67.200000,77.000000,69.30000,75.250000,72.100000,86.450000,522.00000,485.000000,473.000000,603.00000,0.700000


In [11]:
dead_volume = 2.5  # mL, for 20 mL vials

print(f"For 3 mm aspiration height, dead volume of 20 mL vials is around {dead_volume} mL") 

round(transfer[['Drug', 'SL_1', 'SL_2', 'SL_3', 'LL_1', 'LL_2', 'LL_3', 'P_1', 'P_2', 'P_3', 'S_1', 'S_2', 'S_3', 'Water', 'Solvent']].sum()/1000, 2) + dead_volume

For 3 mm aspiration height, dead volume of 20 mL vials is around 2.5 mL


Drug       3.06
SL_1       3.06
SL_2       3.07
SL_3       3.05
LL_1       3.04
LL_2       3.06
LL_3       3.07
P_1        3.05
P_2        3.08
P_3        3.06
S_1        6.61
S_2        6.38
S_3        6.59
Water      6.42
Solvent    2.50
dtype: float64

In [12]:
# file path
ax_client_design_path = "optimizer/optimizer_design_" + str(iteration) + ".json"
unlabeled_trial_path = "data/unlabeled_" + str(iteration) + ".xlsx"
unlabeled_trial_processed_path = "data/unlabeled_processed_" + str(iteration) + ".xlsx"
transfer_path = 'data/transfer_' + str(iteration) + '.xlsx'


In [13]:
ax_client_design.save_to_json_file(ax_client_design_path)

unlabeled_trial.to_excel(unlabeled_trial_path, index=False)

unlabeled_trial_processed.to_excel(unlabeled_trial_processed_path, index=False)

transfer.to_excel(transfer_path, index = False)

# Generate new protocol

In [14]:
# Paths
template_protocol = '../../templates/OT_protocol_template.py'

output_script_path_1 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_1.py'
output_script_path_2 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_2.py'



In [15]:
# Update the script with Excel data
updated_script = calc.update_transfer_script(template_protocol, transfer_path, output_script_path_1, output_script_path_2) 


In [16]:
updated_script

('protocol/iteration_0_OT_2_protocol_1.py',
 'protocol/iteration_0_OT_2_protocol_2.py')

# Perform experiment

# Results

In [17]:
results = calc.parameters_and_complexity(iteration)
results

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Complexity,Complexity_STD
0,0,0.096,0.120,0.100,0.064,0.192,0.016,0.020,0.068,0.170,0.152,0.259,0.257,0.236,0.248,12,0
1,1,0.167,0.068,0.090,0.151,0.090,0.117,0.140,0.095,0.047,0.036,0.522,0.058,0.420,0.000,12,0
2,2,0.119,0.180,0.030,0.056,0.047,0.220,0.063,0.215,0.005,0.065,0.101,0.190,0.373,0.335,12,0
3,3,0.017,0.037,0.171,0.141,0.141,0.070,0.116,0.039,0.122,0.145,0.343,0.373,0.065,0.219,12,0
4,4,0.049,0.196,0.158,0.112,0.112,0.051,0.103,0.162,0.046,0.011,0.052,0.366,0.464,0.118,12,0
5,5,0.176,0.025,0.061,0.000,0.000,0.142,0.163,0.025,0.160,0.247,0.300,0.173,0.086,0.441,10,0
6,6,0.119,0.117,0.139,0.142,0.047,0.123,0.019,0.073,0.140,0.081,0.375,0.056,0.379,0.190,12,0
7,7,0.069,0.058,0.024,0.085,0.157,0.089,0.198,0.131,0.099,0.089,0.166,0.363,0.175,0.296,12,0
8,8,0.057,0.166,0.198,0.020,0.077,0.059,0.083,0.174,0.028,0.139,0.245,0.485,0.077,0.194,12,0
9,9,0.155,0.028,0.004,0.114,0.172,0.185,0.137,0.004,0.137,0.064,0.347,0.044,0.284,0.324,12,0


In [18]:
size_raw = calc.size_raw(iteration)
size_raw

,Data Quality,Item,Size,PD Index
0,Caution,A1,112.566329,0.570796
1,Caution,A2,128.899606,0.570796
2,Caution,A3,126.422722,0.570796
3,Caution,A4,355.207806,0.570796
4,Caution,A5,327.343766,0.558857
5,Caution,A6,336.233634,0.570796
6,Good,B1,277.240457,0.288152
7,Good,B2,273.295928,0.305094
8,Good,B3,257.260206,0.224294
9,Good,B4,230.452422,0.453789


In [19]:
size_processed = calc.process_formulations(size_raw)
size_processed

,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,Formulation 1,1000.000000,0.000000,1.000000,0.000000,0
1,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0
2,Formulation 3,269.265531,10.582329,0.272514,0.042610,1
3,Formulation 4,230.770418,4.715827,0.397357,0.052035,1
4,Formulation 5,289.427392,18.167266,0.388666,0.042201,1
5,Formulation 6,231.139084,9.360171,0.279740,0.039277,1
6,Formulation 7,310.494496,8.462511,0.456674,0.056306,1
7,Formulation 8,213.293905,1.030912,0.213048,0.050725,1
8,Formulation 9,221.950313,14.840261,0.368310,0.015829,1
9,Formulation 10,1000.000000,0.000000,1.000000,0.000000,0


In [20]:
results_final = pd.concat([results, size_processed], axis=1)
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,S_3,Water,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,0,0.096,0.120,0.100,0.064,0.192,0.016,0.020,0.068,0.170,...,0.236,0.248,12,0,Formulation 1,1000.000000,0.000000,1.000000,0.000000,0
1,1,0.167,0.068,0.090,0.151,0.090,0.117,0.140,0.095,0.047,...,0.420,0.000,12,0,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0
2,2,0.119,0.180,0.030,0.056,0.047,0.220,0.063,0.215,0.005,...,0.373,0.335,12,0,Formulation 3,269.265531,10.582329,0.272514,0.042610,1
3,3,0.017,0.037,0.171,0.141,0.141,0.070,0.116,0.039,0.122,...,0.065,0.219,12,0,Formulation 4,230.770418,4.715827,0.397357,0.052035,1
4,4,0.049,0.196,0.158,0.112,0.112,0.051,0.103,0.162,0.046,...,0.464,0.118,12,0,Formulation 5,289.427392,18.167266,0.388666,0.042201,1
5,5,0.176,0.025,0.061,0.000,0.000,0.142,0.163,0.025,0.160,...,0.086,0.441,10,0,Formulation 6,231.139084,9.360171,0.279740,0.039277,1
6,6,0.119,0.117,0.139,0.142,0.047,0.123,0.019,0.073,0.140,...,0.379,0.190,12,0,Formulation 7,310.494496,8.462511,0.456674,0.056306,1
7,7,0.069,0.058,0.024,0.085,0.157,0.089,0.198,0.131,0.099,...,0.175,0.296,12,0,Formulation 8,213.293905,1.030912,0.213048,0.050725,1
8,8,0.057,0.166,0.198,0.020,0.077,0.059,0.083,0.174,0.028,...,0.077,0.194,12,0,Formulation 9,221.950313,14.840261,0.368310,0.015829,1
9,9,0.155,0.028,0.004,0.114,0.172,0.185,0.137,0.004,0.137,...,0.284,0.324,12,0,Formulation 10,1000.000000,0.000000,1.000000,0.000000,0


In [21]:
# µg/mL
results_final['Solu'] = results_final['Drug'] * 600 / 0.3
results_final['Solu_STD'] = 0

results_final.loc[results_final['Formulation Quality'] == 0, 'Solu'] = 0
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,0,0.096,0.120,0.100,0.064,0.192,0.016,0.020,0.068,0.170,...,12,0,Formulation 1,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
1,1,0.167,0.068,0.090,0.151,0.090,0.117,0.140,0.095,0.047,...,12,0,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
2,2,0.119,0.180,0.030,0.056,0.047,0.220,0.063,0.215,0.005,...,12,0,Formulation 3,269.265531,10.582329,0.272514,0.042610,1,238.0,0
3,3,0.017,0.037,0.171,0.141,0.141,0.070,0.116,0.039,0.122,...,12,0,Formulation 4,230.770418,4.715827,0.397357,0.052035,1,34.0,0
4,4,0.049,0.196,0.158,0.112,0.112,0.051,0.103,0.162,0.046,...,12,0,Formulation 5,289.427392,18.167266,0.388666,0.042201,1,98.0,0
5,5,0.176,0.025,0.061,0.000,0.000,0.142,0.163,0.025,0.160,...,10,0,Formulation 6,231.139084,9.360171,0.279740,0.039277,1,352.0,0
6,6,0.119,0.117,0.139,0.142,0.047,0.123,0.019,0.073,0.140,...,12,0,Formulation 7,310.494496,8.462511,0.456674,0.056306,1,238.0,0
7,7,0.069,0.058,0.024,0.085,0.157,0.089,0.198,0.131,0.099,...,12,0,Formulation 8,213.293905,1.030912,0.213048,0.050725,1,138.0,0
8,8,0.057,0.166,0.198,0.020,0.077,0.059,0.083,0.174,0.028,...,12,0,Formulation 9,221.950313,14.840261,0.368310,0.015829,1,114.0,0
9,9,0.155,0.028,0.004,0.114,0.172,0.185,0.137,0.004,0.137,...,12,0,Formulation 10,1000.000000,0.000000,1.000000,0.000000,0,0.0,0


In [22]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,1.000000
1,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,1.000000
2,1,0.269266,0.010582,0.119,0.0,0.272514,0.042610,1.000000
3,1,0.230770,0.004716,0.017,0.0,0.397357,0.052035,1.000000
4,1,0.289427,0.018167,0.049,0.0,0.388666,0.042201,1.000000
5,1,0.231139,0.009360,0.176,0.0,0.279740,0.039277,0.833333
6,1,0.310494,0.008463,0.119,0.0,0.456674,0.056306,1.000000
7,1,0.213294,0.001031,0.069,0.0,0.213048,0.050725,1.000000
8,1,0.221950,0.014840,0.057,0.0,0.368310,0.015829,1.000000
9,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,1.000000


In [23]:
labeled_data_path = 'data/labeled_' + str(iteration) + '.xlsx'
labeled_norm_path = 'data/labeled_norm_' + str(iteration) + '.xlsx'


In [24]:
results_final.to_excel(labeled_data_path, index=False)
results_normalized.to_excel(labeled_norm_path, index=False)

# Load the labeled data to update the optimizer

In [25]:
labeled_norm = pd.read_excel(labeled_norm_path)
labeled_norm.head()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,0,0.096,0.120,0.100,0.064,0.192,0.016,0.020,0.068,0.170,...,1.0,0,Formulation 1,1.000000,0.000000,1.000000,0.000000,0,0.000,0
1,1,0.167,0.068,0.090,0.151,0.090,0.117,0.140,0.095,0.047,...,1.0,0,Formulation 2,1.000000,0.000000,1.000000,0.000000,0,0.000,0
2,2,0.119,0.180,0.030,0.056,0.047,0.220,0.063,0.215,0.005,...,1.0,0,Formulation 3,0.269266,0.010582,0.272514,0.042610,1,0.119,0
3,3,0.017,0.037,0.171,0.141,0.141,0.070,0.116,0.039,0.122,...,1.0,0,Formulation 4,0.230770,0.004716,0.397357,0.052035,1,0.017,0
4,4,0.049,0.196,0.158,0.112,0.112,0.051,0.103,0.162,0.046,...,1.0,0,Formulation 5,0.289427,0.018167,0.388666,0.042201,1,0.049,0


In [26]:
ax_client_design = AxClient.load_from_json_file(ax_client_design_path)


In [27]:
loaded_optimizer_path = "optimizer/optimizer_load_" + str(iteration) + ".json"

In [28]:
loaded_ax_client = sdl.load_labeled_data(ax_client_design, labeled_norm_path)
loaded_ax_client.save_to_json_file(loaded_optimizer_path)

[INFO 08-02 10:27:58] ax.service.ax_client: Completed trial 0 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-02 10:27:58] ax.service.ax_client: Completed trial 1 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-02 10:27:58] ax.service.ax_client: Completed trial 2 with data: {'Solu': (0.119, 0), 'Size': (0.269266, 0.010582), 'PDI': (0.272514, 0.04261)}.
[INFO 08-02 10:27:58] ax.service.ax_client: Completed trial 3 with data: {'Solu': (0.017, 0), 'Size': (0.23077, 0.004716), 'PDI': (0.397357, 0.052035)}.
[INFO 08-02 10:27:58] ax.service.ax_client: Completed trial 4 with data: {'Solu': (0.049, 0), 'Size': (0.289427, 0.018167), 'PDI': (0.388666, 0.042201)}.
[INFO 08-02 10:27:58] ax.service.ax_client: Completed trial 5 with data: {'Solu': (0.176, 0), 'Size': (0.231139, 0.00936), 'PDI': (0.27974, 0.039277)}.
[INFO 08-02 10:27:58] ax.service.ax_client: Completed trial 6 with data: {'Solu': (0.119, 0), 'Size': (0.310494, 0.008463), 